In [25]:
from DiffFlatQuad.robot import PlanerQuadrotor
import sympy as sp

In [26]:
robot = PlanerQuadrotor(rendering=False)

In [27]:
robot.getSymbolicF()

Matrix([
[x2(t)],
[    0],
[x4(t)],
[   -g],
[x6(t)],
[    0]])

In [28]:
robot.getSymbolicG()

Matrix([
[            0,   0],
[-sin(x5(t))/m,   0],
[            0,   0],
[ cos(x5(t))/m,   0],
[            0,   0],
[            0, 1/J]])

In [29]:
robot.symbolic_state

Matrix([
[x1(t)],
[x2(t)],
[x3(t)],
[x4(t)],
[x5(t)],
[x6(t)]])

In [30]:
robot.symbolic_input

Matrix([
[u1(t)],
[u2(t)]])

In [31]:
f = robot.getSymbolicF()
g = robot.getSymbolicG()
x = robot.symbolic_state
g1 = g[:,0]
g2 = g[:,1]
t = sp.symbols('t')


In [32]:
def lieDerivative(a, b, x):
    """ 
    returns the L_a(b)
    """
    return b.jacobian(x)*a

def lieBracket(a,b,x):
    return lieDerivative(a, b, x)-lieDerivative(b, a, x)

In [33]:
def isInDist(dist, vec):
    """
    Is vec in distribution dist?
    """
    d = dist[0].copy()
    for i in range(len(dist)-1):
        d = d.row_join(dist[i+1])
    rank1 = d.rank()
    d = d.row_join(vec)
    rank2 = d.rank()
    if rank2 > rank1:
        return False
    else:
        return True
    
def getDistRank(dist):
    """
    return the rank of distribution spanned by list of vecs in dist
    """
    d = dist[0].copy()
    for i in range(len(dist)-1):
        d = d.row_join(dist[i+1])
    return d.rank()

def dist2Matrix(dist):
    """
    return the rank of distribution spanned by list of vecs in dist
    """
    d = dist[0].copy()
    for i in range(len(dist)-1):
        d = d.row_join(dist[i+1])
    return d

def getSmallestInvariantDistribution(dist, vec_fields, x):
    """
    Returns the smallest distribution invariant to vectors in the 
    vec_fields list and containing distribution spanned by the 
    vector fields in dist list. 
    """
    running = True
    result = []
    result +=dist
    while running:
        added_something = False
        for vec1 in result:
            for vec2 in vec_fields:
                vec3 = lieBracket(vec1, vec2, x)
                if not isInDist(result, vec3):
                    result +=[vec3]
                    added_something = True
        if not added_something:
            running = False
    
    return result
                    
        

In [34]:
result = getSmallestInvariantDistribution([g1,g2], [f,g1,g2],x)
len(result)

6

In [36]:
dist= dist2Matrix(result)
dist


Matrix([
[            0,   0,      -sin(x5(t))/m,   0,   2*x6(t)*cos(x5(t))/m,                 0],
[-sin(x5(t))/m,   0, x6(t)*cos(x5(t))/m,   0,  x6(t)**2*sin(x5(t))/m, -cos(x5(t))/(J*m)],
[            0,   0,       cos(x5(t))/m,   0,   2*x6(t)*sin(x5(t))/m,                 0],
[ cos(x5(t))/m,   0, x6(t)*sin(x5(t))/m,   0, -x6(t)**2*cos(x5(t))/m, -sin(x5(t))/(J*m)],
[            0,   0,                  0, 1/J,                      0,                 0],
[            0, 1/J,                  0,   0,                      0,                 0]])

In [37]:

from sympy import Matrix, print_latex

print_latex(dist)

\left[\begin{matrix}0 & 0 & - \frac{\sin{\left(\operatorname{x_{5}}{\left(t \right)} \right)}}{m} & 0 & \frac{2 \operatorname{x_{6}}{\left(t \right)} \cos{\left(\operatorname{x_{5}}{\left(t \right)} \right)}}{m} & 0\\- \frac{\sin{\left(\operatorname{x_{5}}{\left(t \right)} \right)}}{m} & 0 & \frac{\operatorname{x_{6}}{\left(t \right)} \cos{\left(\operatorname{x_{5}}{\left(t \right)} \right)}}{m} & 0 & \frac{\operatorname{x_{6}}^{2}{\left(t \right)} \sin{\left(\operatorname{x_{5}}{\left(t \right)} \right)}}{m} & - \frac{\cos{\left(\operatorname{x_{5}}{\left(t \right)} \right)}}{J m}\\0 & 0 & \frac{\cos{\left(\operatorname{x_{5}}{\left(t \right)} \right)}}{m} & 0 & \frac{2 \operatorname{x_{6}}{\left(t \right)} \sin{\left(\operatorname{x_{5}}{\left(t \right)} \right)}}{m} & 0\\\frac{\cos{\left(\operatorname{x_{5}}{\left(t \right)} \right)}}{m} & 0 & \frac{\operatorname{x_{6}}{\left(t \right)} \sin{\left(\operatorname{x_{5}}{\left(t \right)} \right)}}{m} & 0 & - \frac{\operatorname{x_{6}}^

In [22]:
print_latex(f)

\left[\begin{matrix}\operatorname{x_{2}}{\left(t \right)}\\0\\\operatorname{x_{4}}{\left(t \right)}\\- g\\\operatorname{x_{6}}{\left(t \right)}\\0\end{matrix}\right]
